In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# Hyperparameters
num_epochs = 40
num_classes = 2
batch_size = 50
learning_rate = 0.001

# C:\Users\ottoa\UD Classes\Cyber Security Research
DATA_PATH = r'C:\\Users\ottoa\UD Classes\Cyber Security Research\\'
# MODEL_STORE_PATH = 'C:\\Users\Chris\PyTorch\pytorch_models\\'

In [133]:
#try to import the 30DB radio ML data
import numpy as np

from scipy.io import loadmat

x = loadmat(DATA_PATH + 'nsl_kdd_pytorch.mat')

sorted(x.keys())

x_data = x['kdd_data_set']

kdd_42a = x_data[:,0:19]
kdd_42b = x_data[:,20:-1]

kdd_42 = np.concatenate((kdd_42a,kdd_42b),1)

print(kdd_42.shape)

max_row = kdd_42.max(0)
max_grid = np.tile(max_row,(125973,1))

print(max_grid.shape)
print(max_grid)

kdd_norm = kdd_42/max_grid




y_labels = x['kdd_attack_bipolar']
new_labels = y_labels - 1

# new_labels = np.argmax(y_labels, axis=0)

print(new_labels.shape)
print(new_labels)


# print(x)


(125973, 39)
(125973, 39)
[[4.2908e+04 3.0000e+00 7.0000e+01 ... 1.0000e+00 1.0000e+00 1.0000e+00]
 [4.2908e+04 3.0000e+00 7.0000e+01 ... 1.0000e+00 1.0000e+00 1.0000e+00]
 [4.2908e+04 3.0000e+00 7.0000e+01 ... 1.0000e+00 1.0000e+00 1.0000e+00]
 ...
 [4.2908e+04 3.0000e+00 7.0000e+01 ... 1.0000e+00 1.0000e+00 1.0000e+00]
 [4.2908e+04 3.0000e+00 7.0000e+01 ... 1.0000e+00 1.0000e+00 1.0000e+00]
 [4.2908e+04 3.0000e+00 7.0000e+01 ... 1.0000e+00 1.0000e+00 1.0000e+00]]
(125973, 1)
[[0]
 [0]
 [1]
 ...
 [0]
 [1]
 [0]]


In [134]:


# print(x_data)
print(y_labels.shape)
print(kdd_norm.shape)

# x_data2 = np.swapaxes(x_data,0,2)
y_labels2 = np.swapaxes(new_labels,0,1)

# print(y_labels2.shape)
print(kdd_norm.shape)

# tensor_x = torch.stack([torch.Tensor(i) for i in x_data]) # transform to torch tensors
# tensor_y = torch.stack([torch.Tensor(i) for i in y_labels])

tensor_x = torch.from_numpy(kdd_norm).float()
tensor_y = torch.from_numpy(y_labels2).long()




# Your image is [h, w, 3] where 3 means the rgb channel, and pytorch expects [b, 3, h, w] where b is batch size.
reshaped = tensor_x.permute(0, 1).unsqueeze(1)
reshaped = reshaped.unsqueeze(3)

print(reshaped.shape)

# print(tensor_x)

tensor_y = tensor_y.squeeze_()

print(tensor_y)

my_dataset = torch.utils.data.TensorDataset(reshaped,tensor_y) # create your datset
train_loader = torch.utils.data.DataLoader(my_dataset, batch_size=batch_size, shuffle=True) # create your dataloader

(125973, 1)
(125973, 39)
(125973, 39)
torch.Size([125973, 1, 39, 1])
tensor([0, 0, 1,  ..., 0, 1, 0])


In [135]:
# # transforms to apply to the data
# trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# # MNIST dataset
# train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True)
# test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

In [136]:
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [141]:
# Create the Model

import torch.nn as nn

#Edit the following:
filter_size = 2500

conv1_features = 1
conv2_features = 2


pad_size = int((filter_size - 1)/2)
#Stop Edit

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, conv1_features, kernel_size=(filter_size,1), stride=1, padding=(pad_size,0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(conv1_features, conv2_features, kernel_size=(filter_size,1), stride=1, padding=(pad_size,0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(conv2_features*9, 50)
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


In [142]:
#Training

model = ConvNet()
print(model)

from torchsummary import summary
summary(model,input_size=(1,39,1))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 1, kernel_size=(2500, 1), stride=(1, 1), padding=(1249, 0))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(1, 2, kernel_size=(2500, 1), stride=(1, 1), padding=(1249, 0))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5)
  (fc1): Linear(in_features=18, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=2, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 1, 38, 1]           2,501
              ReLU-2             [-1, 1, 38, 1]               0
         MaxPool2d-3             [-1, 1, 19, 1]               0
            Conv2d-4             [-1, 2, 18, 1]           5,002
              ReLU-5             [-1, 2, 1

In [143]:
# Train the model
# aaa = torch.max(labels, 1)[1]
# print(labels.shape)
# print(aaa.shape)

total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
#         labels = labels.long()
#         labels = labels.squeeze(0)
#         labels = labels.squeeze_()
#         print(labels.shape())
#         labels = labels.to(dtype=torch.int64)

        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/40], Step [100/2520], Loss: 0.6822, Accuracy: 62.00%
Epoch [1/40], Step [200/2520], Loss: 0.6811, Accuracy: 60.00%
Epoch [1/40], Step [300/2520], Loss: 0.6864, Accuracy: 56.00%
Epoch [1/40], Step [400/2520], Loss: 0.7023, Accuracy: 44.00%
Epoch [1/40], Step [500/2520], Loss: 0.6837, Accuracy: 60.00%
Epoch [1/40], Step [600/2520], Loss: 0.6825, Accuracy: 58.00%
Epoch [1/40], Step [700/2520], Loss: 0.6924, Accuracy: 52.00%
Epoch [1/40], Step [800/2520], Loss: 0.6939, Accuracy: 50.00%
Epoch [1/40], Step [900/2520], Loss: 0.6806, Accuracy: 60.00%
Epoch [1/40], Step [1000/2520], Loss: 0.6794, Accuracy: 66.00%
Epoch [1/40], Step [1100/2520], Loss: 0.6767, Accuracy: 62.00%
Epoch [1/40], Step [1200/2520], Loss: 0.6822, Accuracy: 58.00%
Epoch [1/40], Step [1300/2520], Loss: 0.6885, Accuracy: 56.00%
Epoch [1/40], Step [1400/2520], Loss: 0.6906, Accuracy: 54.00%
Epoch [1/40], Step [1500/2520], Loss: 0.6725, Accuracy: 66.00%
Epoch [1/40], Step [1600/2520], Loss: 0.6809, Accuracy: 66.00%
E

Epoch [6/40], Step [700/2520], Loss: 0.6875, Accuracy: 56.00%
Epoch [6/40], Step [800/2520], Loss: 0.6945, Accuracy: 50.00%
Epoch [6/40], Step [900/2520], Loss: 0.6733, Accuracy: 68.00%
Epoch [6/40], Step [1000/2520], Loss: 0.6989, Accuracy: 48.00%
Epoch [6/40], Step [1100/2520], Loss: 0.6844, Accuracy: 62.00%
Epoch [6/40], Step [1200/2520], Loss: 0.6848, Accuracy: 60.00%
Epoch [6/40], Step [1300/2520], Loss: 0.6963, Accuracy: 50.00%
Epoch [6/40], Step [1400/2520], Loss: 0.6983, Accuracy: 48.00%
Epoch [6/40], Step [1500/2520], Loss: 0.6974, Accuracy: 48.00%
Epoch [6/40], Step [1600/2520], Loss: 0.6964, Accuracy: 48.00%
Epoch [6/40], Step [1700/2520], Loss: 0.7042, Accuracy: 44.00%
Epoch [6/40], Step [1800/2520], Loss: 0.6900, Accuracy: 54.00%
Epoch [6/40], Step [1900/2520], Loss: 0.6866, Accuracy: 56.00%
Epoch [6/40], Step [2000/2520], Loss: 0.6929, Accuracy: 52.00%
Epoch [6/40], Step [2100/2520], Loss: 0.6928, Accuracy: 52.00%
Epoch [6/40], Step [2200/2520], Loss: 0.6929, Accuracy: 52

Epoch [11/40], Step [1300/2520], Loss: 0.6843, Accuracy: 60.00%
Epoch [11/40], Step [1400/2520], Loss: 0.6857, Accuracy: 60.00%
Epoch [11/40], Step [1500/2520], Loss: 0.6888, Accuracy: 56.00%
Epoch [11/40], Step [1600/2520], Loss: 0.6902, Accuracy: 54.00%
Epoch [11/40], Step [1700/2520], Loss: 0.6899, Accuracy: 54.00%
Epoch [11/40], Step [1800/2520], Loss: 0.7051, Accuracy: 46.00%
Epoch [11/40], Step [1900/2520], Loss: 0.6965, Accuracy: 50.00%
Epoch [11/40], Step [2000/2520], Loss: 0.6806, Accuracy: 60.00%
Epoch [11/40], Step [2100/2520], Loss: 0.6829, Accuracy: 60.00%
Epoch [11/40], Step [2200/2520], Loss: 0.6928, Accuracy: 52.00%
Epoch [11/40], Step [2300/2520], Loss: 0.6948, Accuracy: 50.00%
Epoch [11/40], Step [2400/2520], Loss: 0.6825, Accuracy: 60.00%
Epoch [11/40], Step [2500/2520], Loss: 0.6708, Accuracy: 66.00%
Epoch [12/40], Step [100/2520], Loss: 0.6965, Accuracy: 50.00%
Epoch [12/40], Step [200/2520], Loss: 0.6900, Accuracy: 54.00%
Epoch [12/40], Step [300/2520], Loss: 0.69

Epoch [16/40], Step [1700/2520], Loss: 0.6893, Accuracy: 56.00%
Epoch [16/40], Step [1800/2520], Loss: 0.6821, Accuracy: 62.00%
Epoch [16/40], Step [1900/2520], Loss: 0.6948, Accuracy: 50.00%
Epoch [16/40], Step [2000/2520], Loss: 0.6841, Accuracy: 60.00%
Epoch [16/40], Step [2100/2520], Loss: 0.6774, Accuracy: 64.00%
Epoch [16/40], Step [2200/2520], Loss: 0.7014, Accuracy: 46.00%
Epoch [16/40], Step [2300/2520], Loss: 0.6874, Accuracy: 56.00%
Epoch [16/40], Step [2400/2520], Loss: 0.7188, Accuracy: 34.00%
Epoch [16/40], Step [2500/2520], Loss: 0.6900, Accuracy: 54.00%
Epoch [17/40], Step [100/2520], Loss: 0.6926, Accuracy: 52.00%
Epoch [17/40], Step [200/2520], Loss: 0.6927, Accuracy: 52.00%
Epoch [17/40], Step [300/2520], Loss: 0.6946, Accuracy: 50.00%
Epoch [17/40], Step [400/2520], Loss: 0.6933, Accuracy: 52.00%
Epoch [17/40], Step [500/2520], Loss: 0.6877, Accuracy: 56.00%
Epoch [17/40], Step [600/2520], Loss: 0.6994, Accuracy: 46.00%
Epoch [17/40], Step [700/2520], Loss: 0.6842, 

Epoch [21/40], Step [2100/2520], Loss: 0.6792, Accuracy: 62.00%
Epoch [21/40], Step [2200/2520], Loss: 0.6819, Accuracy: 60.00%
Epoch [21/40], Step [2300/2520], Loss: 0.6773, Accuracy: 62.00%
Epoch [21/40], Step [2400/2520], Loss: 0.6899, Accuracy: 54.00%
Epoch [21/40], Step [2500/2520], Loss: 0.6851, Accuracy: 58.00%
Epoch [22/40], Step [100/2520], Loss: 0.6970, Accuracy: 48.00%
Epoch [22/40], Step [200/2520], Loss: 0.6955, Accuracy: 50.00%
Epoch [22/40], Step [300/2520], Loss: 0.6899, Accuracy: 54.00%
Epoch [22/40], Step [400/2520], Loss: 0.6738, Accuracy: 66.00%
Epoch [22/40], Step [500/2520], Loss: 0.6927, Accuracy: 52.00%
Epoch [22/40], Step [600/2520], Loss: 0.6928, Accuracy: 52.00%
Epoch [22/40], Step [700/2520], Loss: 0.6834, Accuracy: 58.00%
Epoch [22/40], Step [800/2520], Loss: 0.6855, Accuracy: 58.00%
Epoch [22/40], Step [900/2520], Loss: 0.6877, Accuracy: 56.00%
Epoch [22/40], Step [1000/2520], Loss: 0.6927, Accuracy: 52.00%
Epoch [22/40], Step [1100/2520], Loss: 0.6955, Ac

Epoch [26/40], Step [2500/2520], Loss: 0.6822, Accuracy: 60.00%
Epoch [27/40], Step [100/2520], Loss: 0.6948, Accuracy: 50.00%
Epoch [27/40], Step [200/2520], Loss: 0.6928, Accuracy: 52.00%
Epoch [27/40], Step [300/2520], Loss: 0.6808, Accuracy: 60.00%
Epoch [27/40], Step [400/2520], Loss: 0.6928, Accuracy: 52.00%
Epoch [27/40], Step [500/2520], Loss: 0.6873, Accuracy: 56.00%
Epoch [27/40], Step [600/2520], Loss: 0.6769, Accuracy: 62.00%
Epoch [27/40], Step [700/2520], Loss: 0.6899, Accuracy: 54.00%
Epoch [27/40], Step [800/2520], Loss: 0.6782, Accuracy: 64.00%
Epoch [27/40], Step [900/2520], Loss: 0.6955, Accuracy: 50.00%
Epoch [27/40], Step [1000/2520], Loss: 0.6801, Accuracy: 60.00%
Epoch [27/40], Step [1100/2520], Loss: 0.6927, Accuracy: 52.00%
Epoch [27/40], Step [1200/2520], Loss: 0.6959, Accuracy: 50.00%
Epoch [27/40], Step [1300/2520], Loss: 0.6874, Accuracy: 56.00%
Epoch [27/40], Step [1400/2520], Loss: 0.6768, Accuracy: 62.00%
Epoch [27/40], Step [1500/2520], Loss: 0.6958, Ac

Epoch [32/40], Step [400/2520], Loss: 0.6775, Accuracy: 64.00%
Epoch [32/40], Step [500/2520], Loss: 0.6872, Accuracy: 56.00%
Epoch [32/40], Step [600/2520], Loss: 0.6928, Accuracy: 52.00%
Epoch [32/40], Step [700/2520], Loss: 0.7025, Accuracy: 46.00%
Epoch [32/40], Step [800/2520], Loss: 0.7103, Accuracy: 40.00%
Epoch [32/40], Step [900/2520], Loss: 0.6966, Accuracy: 50.00%
Epoch [32/40], Step [1000/2520], Loss: 0.6955, Accuracy: 50.00%
Epoch [32/40], Step [1100/2520], Loss: 0.7025, Accuracy: 46.00%
Epoch [32/40], Step [1200/2520], Loss: 0.6900, Accuracy: 54.00%
Epoch [32/40], Step [1300/2520], Loss: 0.6836, Accuracy: 58.00%
Epoch [32/40], Step [1400/2520], Loss: 0.6844, Accuracy: 58.00%
Epoch [32/40], Step [1500/2520], Loss: 0.7025, Accuracy: 44.00%
Epoch [32/40], Step [1600/2520], Loss: 0.6926, Accuracy: 52.00%
Epoch [32/40], Step [1700/2520], Loss: 0.6900, Accuracy: 54.00%
Epoch [32/40], Step [1800/2520], Loss: 0.6958, Accuracy: 50.00%
Epoch [32/40], Step [1900/2520], Loss: 0.6960,

Epoch [37/40], Step [800/2520], Loss: 0.7033, Accuracy: 44.00%
Epoch [37/40], Step [900/2520], Loss: 0.6980, Accuracy: 48.00%
Epoch [37/40], Step [1000/2520], Loss: 0.6755, Accuracy: 64.00%
Epoch [37/40], Step [1100/2520], Loss: 0.6946, Accuracy: 50.00%
Epoch [37/40], Step [1200/2520], Loss: 0.6880, Accuracy: 56.00%
Epoch [37/40], Step [1300/2520], Loss: 0.7056, Accuracy: 42.00%
Epoch [37/40], Step [1400/2520], Loss: 0.6844, Accuracy: 58.00%
Epoch [37/40], Step [1500/2520], Loss: 0.6850, Accuracy: 58.00%
Epoch [37/40], Step [1600/2520], Loss: 0.6930, Accuracy: 52.00%
Epoch [37/40], Step [1700/2520], Loss: 0.6960, Accuracy: 50.00%
Epoch [37/40], Step [1800/2520], Loss: 0.6978, Accuracy: 50.00%
Epoch [37/40], Step [1900/2520], Loss: 0.6828, Accuracy: 60.00%
Epoch [37/40], Step [2000/2520], Loss: 0.6793, Accuracy: 62.00%
Epoch [37/40], Step [2100/2520], Loss: 0.6901, Accuracy: 54.00%
Epoch [37/40], Step [2200/2520], Loss: 0.6873, Accuracy: 56.00%
Epoch [37/40], Step [2300/2520], Loss: 0.7

In [144]:
# Test the model
test_loader = torch.utils.data.DataLoader(my_dataset, batch_size=batch_size, shuffle=False) # create your dataloader

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

# Save the model and plot
torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model_KDD_bin-40E.ckpt')

Test Accuracy of the model on the 10000 test images: 53.458280742698825 %


NameError: name 'MODEL_STORE_PATH' is not defined